## Preambule

In [7]:
# General packages
import numpy as np
import pandas as pd
from tqdm import tqdm
from pathlib import Path
import xarray as xr
import json

# Plotting
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.colors import n_colors

## Parameter

In [8]:
hist_startyear = 1990

## Paths

In [9]:
path_main = Path("X:/user/dekkerm/Projects/ECEMF_T5.2/")
path_data = path_main / "Data" / str(hist_startyear)
path_figs = path_main / "Figures"

## Read data files

In [10]:
xr_total = xr.open_dataset(path_data / "xr_total.nc")
all_regions_iso = np.load(path_data / "all_regions.npy")
all_regions_names = np.load(path_data / "all_regions_names.npy")
all_countries_iso = np.load(path_data / "all_countries.npy", allow_pickle=True)
all_countries_names = np.load(path_data / "all_countries_names.npy", allow_pickle=True)
only_regions_iso = ['CVF', 'G20', "EU", "G7", "NA", "AU", "AF", "SIDS", "LDC", "WORLD"]
only_regions_names = ['Climate Vulnerability Forum','G20',"European Union","G7","Northern America","Australasia","African Group","Small Island Developing States","Least Developed Countries"]

In [11]:
xr_budgets_static = xr.open_dataset(path_data / "xr_budgets_static.nc")
#xr_budgets_linear = xr.open_dataset(path_data / "xr_budgets_linear.nc")
xr_budgets_scenario = xr.open_dataset(path_data / "xr_budgets_scenario.nc")
df_michel = pd.read_excel(Path("X:/user/dekkerm/Data/") / "NDC" / "NDC_comparison.xlsx", sheet_name="Michel")

In [12]:
factor_lulucf = float(np.mean(xr_total.GHG_f_incl.sel(Time=2030)/xr_total.GHG_f.sel(Time=2030)))

In [13]:
df = pd.read_excel(Path("X:/user/dekkerm/Data/") / "UNFCCC_Parties_Groups_noeu.xlsx", sheet_name = "Country groups")
countries_iso = np.array(df["Country ISO Code"])
group_cvf = countries_iso[np.array(df["CVF (24/10/22)"]) == 1]
group_g20 = countries_iso[np.array(df["G20"]) == 1]
group_eu = countries_iso[np.array(df["EU"]) == 1]
group_g7 = countries_iso[np.array(df["G7"]) == 1]
group_na = countries_iso[np.array(df["Northern America"]) == 1]
group_um = countries_iso[np.array(df["Umbrella"]) == 1]
group_au = countries_iso[np.array(df["Australasia"]) == 1]
group_af = countries_iso[np.array(df["African Group"]) == 1]
group_sids = countries_iso[np.array(df["SIDS"]) == 1]
group_ldc = countries_iso[np.array(df["LDC"]) == 1]
group_eg = countries_iso[np.array(df["European Group"]) == 1]
group_hdc = countries_iso[np.array(df["HICO"]) == 1]

In [14]:
df = pd.read_csv(Path("X:/user/dekkerm/Data/") / "AllCountries.csv")
cont_iso = np.array(df['alpha-3']).astype(str)
cont_reg = np.array(df['sub-region']).astype(str)
unireg = np.unique(cont_reg)[:-1]

Find out high/medium/low income

In [15]:
group_mdc = []
for reg in all_countries_iso:
    if reg not in group_hdc and reg not in group_ldc:
        group_mdc.append(reg)

In [16]:
rules = ['GF', 'PC', 'PCC', 'AP', 'GDR', 'ECPC']

In [17]:
high_vals = []
for reg_i, reg in enumerate(group_hdc):
    ar =np.array([xr_budgets_static.sel(Category='C1+C2', ISO = reg)[i] for i in rules])
    #rule = rules[]
    high_vals.append(np.where(ar == np.nanmax(ar))[0][0])
med_vals = []
for reg_i, reg in enumerate(group_mdc):
    ar = np.array([xr_budgets_static.sel(Category='C1+C2', ISO = reg)[i] for i in rules])
    #rule = rules[]
    med_vals.append(np.where(ar == np.nanmax(ar))[0][0])
low_vals = []
for reg_i, reg in enumerate(group_ldc):
    ar = np.array([xr_budgets_static.sel(Category='C1+C2', ISO = reg)[i] for i in rules])
    #rule = rules[]
    low_vals.append(np.where(ar == np.nanmax(ar))[0][0])

## Plot

In [22]:
fig = make_subplots(rows=4, cols=6,
                    specs=[[{'colspan': 3, 'rowspan': 2}, {}, {}, {'colspan': 3, 'rowspan': 2,'type': 'choropleth'}, {}, {}],
                           [{}, {}, {}, {}, {}, {}],
                           [{'colspan': 2}, {}, {'colspan': 2}, {}, {'colspan': 2}, {}],
                           [{'colspan': 2,'type': 'choropleth'}, {}, {'colspan': 2,'type': 'choropleth'}, {}, {'colspan': 2,'type': 'choropleth'}, {}]],
                    horizontal_spacing = 0.02, vertical_spacing=0.06)

fig.update_layout(height=1300,
                  width=1600,
                  template='simple_white')
# PANEL (A), (C), (D), (E)
lw = 4
for ax_i in range(4):
       regs = [['WORLD'], group_hdc, group_mdc, group_ldc][ax_i]
       row = [1, 3, 3, 3][ax_i]
       col = [1, 1, 3, 5][ax_i]
       if ax_i == 0: sl = True
       else: sl=False
       fig.add_trace(go.Scatter(x=np.arange(1990, 2020),
                            y=xr_total.sel(Time=np.arange(1990, 2020), ISO=regs, Category='C1+C2').GHG_p.sum(dim='ISO'),
                            name='Historical emissions',
                            line={'color': 'silver', 'width': lw},
                            mode='lines',
                            showlegend=sl), row, col)
       fig.add_trace(go.Scatter(x=np.arange(2020, 2101),
                            y=xr_total.sel(Time=np.arange(2020, 2101), ISO=regs, Category='C1+C2').GHG_base.sum(dim='ISO'),
                            name='SSP2-baseline',
                            line={'color': 'grey', 'width': lw},
                            mode='lines',
                            showlegend=sl), row, col)
       if ax_i == 0:
              for cat_i, cat in enumerate(['C1+C2', "C3", "C5", "C7"]):
                     fig.add_trace(go.Scatter(x=np.arange(2019, 2101),
                                              y=xr_total.sel(Time=np.arange(2019, 2101), ISO='WORLD', Category=cat).GHG_f,
                                              name="Cost-optimal: "+['1.5 degrees', '2 degrees', '3 degrees', '4 degrees'][cat_i],
                                              line={'color': ['lightsalmon', 'orangered', 'red', 'darkred'][cat_i], 'width': lw},
                                              mode='lines',
                                              showlegend=sl), row, col)
       else:
              for var_i, var in enumerate(rules):
                     fig.add_trace(go.Scatter(x=np.arange(2020, 2101),
                                              y=xr_budgets_scenario.sel(Time=np.arange(2020, 2101), ISO=regs, Category='C1+C2')[var].sum(dim='ISO'),
                                              name=['Grandfathering', 'Per capita', 'Per capita convergence', 'Ability to pay', 'Greenhouse Development Rights', 'Equal Cumulative per capita'][var_i],
                                              line={'color': ['steelblue', 'goldenrod', 'purple', 'forestgreen', 'chocolate', 'violet'][var_i], 'width': lw},
                                              mode='lines',
                                              showlegend=[False, True, False, False][ax_i]), row, col)

              fig.add_trace(
                     go.Choropleth(
                     locations=regs,
                     z = [high_vals, med_vals, low_vals][ax_i-1],
                     locationmode = 'ISO-3',
                     colorscale = ['steelblue', 'goldenrod', 'purple', 'forestgreen', 'chocolate', 'violet'],
                     zmax = 5,
                     zmin = 0,
                     #text = ["EU: "+str(int(t[np.where(regs == "EU")[0][0]]*len(allocs)))+" out of 6"for r_i, r in enumerate(group_eu)],
                     hovertemplate  = '%{text}',
                     name="",
                     marker_line_color='black', 
                     marker_line_width=0.75,
                     showscale=False
                     ), row+1, col)
       
       fig.add_trace(go.Scatter(x=[2030],
                            y=[xr_total.sel(Time=2030, ISO=regs, Category='C1+C2').NDC_l.sum(dim='ISO')],
                            name='NDCs',
                            line={'color': 'black', 'width': lw},
                            mode='lines+markers',
                            marker={'size': 12, 'symbol': 'triangle-down'},
                            showlegend=False), row, col)
       
       fig.add_trace(go.Scatter(x=[2030],
                            y=[xr_total.sel(Time=2030, ISO=regs, Category='C1+C2').NDC_h.sum(dim='ISO')],
                            name='NDCs',
                            line={'color': 'black', 'width': lw},
                            mode='lines+markers',
                            marker={'size': 12, 'symbol': 'triangle-up'},
                            showlegend=False), row, col)
       
       fig.add_trace(go.Scatter(x=[2030, 2030],
                            y=[xr_total.sel(Time=2030, ISO=regs, Category='C1+C2').NDC_l.sum(dim='ISO'), xr_total.sel(Time=2030, ISO=regs, Category='C1+C2').NDC_h.sum(dim='ISO')],
                            name='NDCs',
                            line={'color': 'black', 'width': lw},
                            mode='lines',
                            marker={'size': 12, 'symbol': 'triangle-up'},
                            showlegend=sl), row, col)

fig.update_layout(legend=dict(
    yanchor="top",
    y=0.76,
    xanchor="left",
    x=0.01,
    font=dict(
        size=11,
        color="black"
    ),
))
fig.update_yaxes(title_text="Global GHG emissions [Gt CO2e/yr]", row=1, col=1)

# PANEL (B)
ndc = (np.array(xr_total.NDC_h.sel(Time=2030, ISO=all_countries_iso))+np.array(xr_total.NDC_l.sel(Time=2030, ISO=all_countries_iso)))/2
base = np.array(xr_total.GHG_base.sel(Time=2030, ISO=all_countries_iso))
t = (ndc-base)/base
fig.add_trace(
       go.Choropleth(
       locations=all_countries_iso,
       z = t,
       locationmode = 'ISO-3',
       colorscale = 'RdBu_r',
       zmax = 1,
       zmin = -1,
       text = [str(r)+": "+str(np.round(float(t[r_i]*100), 2))+"%" for r_i, r in enumerate(all_countries_iso)],
       hovertemplate  = '%{text}',
       name="",
       marker_line_color='black', 
       marker_line_width=0.6,
       colorbar=dict(len=0.32, x=0.76, y=0.49, orientation='h',
                  tickvals = np.arange(-1, 1.05, 0.5),
                  thickness=15,
                  tickfont={'size': 13},
                  ticktext=['-100%<br>(emission reduction<br>w.r.t. baseline)', '-50%', '0%', '50%', '100%<br>(emission increase<br>w.r.t. baseline)'],
                  ticklabelposition='outside',
    ),
       showscale=True
), 1, 4)

fig.update_geos(showocean=True, oceancolor="aliceblue")
fig.update_geos(showlakes=True, lakecolor="aliceblue")
fig.update_geos(visible=True,
                showlakes=False,
                lakecolor='rgb(255, 255, 255)',
                coastlinecolor='silver',
                projection_type='natural earth',
                showcoastlines=True)
fig.update_geos(visible=True,
                showlakes=True,
                lakecolor='rgb(255, 255, 255)',
                projection_type='natural earth',
                showcoastlines=True, row=1, col=4)

fig.update_yaxes(title_text="GHG emissions [Gt CO2e/yr]", row=3, col=1)
fig.add_annotation(xref='paper', yref='paper',ax=0, align='left', ay=-0, showarrow=False, font=dict(color='black', size=18),
                   y=0.97, x=0.01, text="<b>(a) Global emission trajectories</b>")
fig.add_annotation(xref='paper', yref='paper',ax=0, align='left', ay=-0, showarrow=False, font=dict(color='black', size=18),
                   y=0.97, x=0.90, text="<b>(b) NDC emissions in 2030 w.r.t. baseline</b>")
fig.add_annotation(xref='paper', yref='paper',ax=0, align='left', ay=-0, showarrow=False, font=dict(color='black', size=18),
                   y=0.48, x=0.01, text="<b>(c) High-income countries</b>")
fig.add_annotation(xref='paper', yref='paper',ax=0, align='left', ay=-0, showarrow=False, font=dict(color='black', size=18),
                   y=0.48, x=0.46, text="<b>(d) Medium-income countries</b>")
fig.add_annotation(xref='paper', yref='paper',ax=0, align='left', ay=-0, showarrow=False, font=dict(color='black', size=18),
                   y=0.48, x=0.88, text="<b>(e) Low-income countries</b>")
fig.show()

In [19]:
fig.write_image("X:/user/dekkerm/Projects/ECEMF_T5.2/Figures/Figure_globalassess/Figure_globalassess.svg")
fig.write_image("X:/user/dekkerm/Projects/ECEMF_T5.2/Figures/Figure_globalassess/Figure_globalassess.png", scale=3)

import os
from datetime import date
try:
    os.remove('X:/user/dekkerm/Projects/ECEMF_T5.2/Figures/Figure_globalassess/Figure_globalassess.html')
except:
    3
def html_w(typ):
    return '<html> '+typ+' <p style="font-family: Arial">'

today = date.today()
td = today.strftime("%B %d, %Y")

with open('X:/user/dekkerm/Projects/ECEMF_T5.2/Figures/Figure_globalassess/Figure_globalassess.html', 'a') as f:
    #f.write(html_w('<h1>')+'Figure 1 - Electricity Generation</p></h1>')
    f.write(html_w('<body>')+'Version: '+td+'</p></body>')
    f.write(fig.to_html(full_html=False, include_plotlyjs='cdn'))